# Load datasets

In [44]:
from datasets import load_dataset, Dataset
import re

def get_vi_gsm8k_questions(split="train") -> Dataset:
    data = load_dataset('hllj/vi_gsm8k')[split] 

    def normalize_answer(example):
        # Loại bỏ dấu phẩy trong số và chuyển thành dạng số nguyên
        example['answer'] = re.sub(r'[^\d]', '', example['answer'])
        return example

    data = data.map(normalize_answer) 

    data = data.map(lambda x: { 
        'answer': x['explanation'] + "\n### " + x['answer'],
    })  

    return data  

dataset = get_vi_gsm8k_questions(split = "train")
test_dataset = get_vi_gsm8k_questions(split = "test")

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [45]:
dataset

Dataset({
    features: ['index', 'explanation', 'question', 'answer'],
    num_rows: 7473
})

In [46]:
test_dataset

Dataset({
    features: ['index', 'explanation', 'question', 'answer'],
    num_rows: 1319
})

In [47]:
dataset[0]

{'index': 0,
 'explanation': 'Natalia đã bán 24 kẹp trong tháng 5.\nNatalia đã bán tổng cộng 72 kẹp trong tháng 4 và tháng 5.',
 'question': 'Natalia đã bán kẹp tóc cho 48 người bạn của cô ấy vào tháng 4, và sau đó cô ấy đã bán nửa số lượng kẹp tóc đó vào tháng 5. Natalia đã bán tổng cộng bao nhiêu kẹp tóc trong tháng 4 và tháng 5?',
 'answer': 'Natalia đã bán 24 kẹp trong tháng 5.\nNatalia đã bán tổng cộng 72 kẹp trong tháng 4 và tháng 5.\n### 72'}

# Load model

In [5]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model_name = "unsloth/Qwen2.5-3B-Instruct"

model, _ = FastLanguageModel.from_pretrained(
    model_name,
    device_map="cuda:0",
    dtype = torch.bfloat16,
    max_seq_length=1536,
    load_in_4bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'
# model.enable_input_require_grads() 
# tokenizer.padding_side  = 'left' # Must set to left because of flash attn 2

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [58]:
FastLanguageModel.for_inference(model)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
SYSTEM_PROMPT = \
"""Bạn là một trợ lý toán học.
Hãy suy nghĩ cẩn thận để giải bài toán được cung cấp.
Trình bày từng bước. Sau đó trả lời theo cú pháp:
### number
"""

prompt = "Có bao nhiêu chữ 'r' trong từ 'waterrmelon' ?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256,
    streamer = text_streamer
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

<|im_start|>system
Bạn là một trợ lý toán học.
Hãy suy nghĩ cẩn thận để giải bài toán được cung cấp.
Trình bày từng bước. Sau đó trả lời theo cú pháp:
### number
<|im_end|>
<|im_start|>user
Có bao nhiêu chữ 'r' trong từ 'waterrmelon' ?<|im_end|>
<|im_start|>assistant
Để giải bài toán này, chúng ta sẽ kiểm tra từng chữ cái trong từ 'waterrmelon' xem có chữ 'r' không.

1. Đầu tiên, chúng ta xem từ 'waterrmelon':
   - 'w'
   - 'a'
   - 't'
   - 'e'
   - 'r'
   - 'r'
   - 'm'
   - 'e'
   - 'l'
   - 'o'
   - 'n'

2. Trong từ trên, chúng ta thấy có hai chữ 'r'.

Vậy, số chữ 'r' trong từ 'waterrmelon' là 2.

### 2<|im_end|>


# Build QLoRA config

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    use_rslora = True,  
    loftq_config = None, 
)

print (model.print_trainable_parameters())

Unsloth 2025.4.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


trainable params: 59,867,136 || all params: 3,145,805,824 || trainable%: 1.9031
None


In [14]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

# Preprocessing

In [60]:
def preprocess_batch(batch):
    # Extract fields for the entire batch
    inputs = batch["question"]
    outputs = batch["answer"]

    processed_texts = []
    for input_text, output in zip(inputs, outputs):
        input_text = input_text.strip() if input_text else ""
        output = output.strip()

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": input_text},
            {"role": "assistant", "content": output}
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            # add_generation_prompt=True # comment this for training
        )
        processed_texts.append(text)
    return processed_texts

In [61]:
# data collator for causal LM
from trl import DataCollatorForCompletionOnlyLM

response_template = "<|im_start|>assistant\n"
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template=response_template,
)

# Training

In [62]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(
    # Paths & Datasets
    output_dir="qwen-2.5-3b-instruct-math-qa",    
    
    # Truncation 
    max_seq_length=1536,
    
    per_device_train_batch_size=6,       
    per_device_eval_batch_size=6,
    gradient_accumulation_steps=2,

    # Optimization & LR Scheduling
    learning_rate=1e-4,
    weight_decay=0.05,
    num_train_epochs=2,
    lr_scheduler_type="cosine",
    warmup_steps=10,

    # Evaluation / Checkpoint
    eval_strategy="steps",              
    save_strategy="steps",              
    logging_strategy="steps",
    eval_steps=200,       
    save_steps=200,
    save_total_limit=1,

    # Best‑model selection
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    gradient_checkpointing=True,
    optim ="paged_adamw_8bit",
    report_to="none"
)

In [63]:
trainer = SFTTrainer(
    model=model,                         
    train_dataset=dataset,
    eval_dataset=test_dataset,
    args=sft_config,                     
    processing_class=tokenizer,
    formatting_func=preprocess_batch,
    data_collator=data_collator,
    dataset_num_proc=1
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/7473 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [64]:
history = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 2 | Total steps = 1,246
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 2 x 1) = 12
 "-____-"     Trainable parameters = 59,867,136/3,000,000,000 (2.00% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
200,0.447700,0.599701
400,0.535400,0.574737
600,0.583100,0.559330
800,0.375500,0.572599
1000,0.363700,0.566545
1200,0.383000,0.565581


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [65]:
metrics = trainer.evaluate(trainer.eval_dataset)

In [67]:
# calculate perplexity
import math
perplexity = math.exp(metrics["eval_loss"])
perplexity

1.7494996062448478

In [68]:
torch.cuda.empty_cache()

In [ ]:
# push to hub
trainer.push_to_hub("binhphap5/qwen-2.5-3b-instruct-math-qa")

In [69]:
input = 'Lan có 2 cây bút, hôm sau Lan mua thêm 1 cây bút nữa, vậy Lan có tổng cộng bao nhiêu cây bút?'

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": input},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True # comment this for training
)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>system
Bạn là một trợ lý toán học.
Hãy suy nghĩ cẩn thận để giải bài toán được cung cấp.
Trình bày từng bước. Sau đó trả lời theo cú pháp:
### number
<|im_end|>
<|im_start|>user
Lan có 2 cây bút, hôm sau Lan mua thêm 1 cây bút nữa, vậy Lan có tổng cộng bao nhiêu cây bút?<|im_end|>
<|im_start|>assistant
Lan có 2 cây bút + 1 cây bút = 3 cây bút.
### 3<|im_end|>


In [70]:
torch.cuda.empty_cache()

In [1]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model_name = "qwen-2.5-3b-instruct-math-qa/checkpoint-2400"

model, _ = FastLanguageModel.from_pretrained(
    model_name,
    device_map="cuda:0",
    dtype = torch.bfloat16,
    max_seq_length=2048,
    load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'
# model.enable_input_require_grads() 
# tokenizer.padding_side  = 'left' # Must set to left because of flash attn 2

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.4.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


# Batch generation helper function

In [88]:
import tqdm
import re

def extract_reasoning_and_answer(text):
    """
    Given the full output text, extract the reasoning and the final answer robustly.
    Handles noisy generation after the answer.
    """
    answer_markers = ["###"]
    
    marker_found = None
    for marker in answer_markers:
        if marker in text:
            marker_found = marker
            break
    
    if marker_found:
        parts = text.split(marker_found, 1)
        reasoning = parts[0].strip()
        answer = parts[1].strip()
                
    else:
        # fallback nếu không tìm thấy marker
        reasoning = text.strip()
        answer = ""
    
    return reasoning, answer

def generate_batch_responses(input_texts, model, tokenizer, batch_size=20, max_length=768):
    tokenizer.padding_side = "left"
    all_outputs = []
    for i in tqdm(range(0, len(input_texts), batch_size), desc="Generating responses"):
        batch_inputs = input_texts[i:i + batch_size]
        
        # Apply chat template for each input in the batch
        batch_prompts = []
        for text in batch_inputs:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": text},
            ]
            prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True  # Comment this for training
            )
            batch_prompts.append(prompt)
        
        # Tokenize all prompts in the batch
        batch_encodings = tokenizer(
            batch_prompts, 
            return_tensors="pt", 
            padding=True, 
            truncation=True
        ).to(model.device)
        
        # Generate for all inputs in batch
        with torch.no_grad():
            FastLanguageModel.for_inference(model)
            outputs = model.generate(
                **batch_encodings,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        # Decode all outputs
        decoded_outputs = [
            tokenizer.decode(output, skip_special_tokens=True)
            for output in outputs
        ]
        
        # Extract responses
        responses = [
            output.split("\nassistant\n")[-1].strip()
            for output in decoded_outputs
        ]

        # Extract reasoning và answer
        for response in responses:
            reasoning, answer = extract_reasoning_and_answer(response)
            all_outputs.append({
                "reasoning": reasoning,
                "answer": answer,
                "raw": response
            })
        
        torch.cuda.empty_cache()
        
    return all_outputs

In [89]:
tokenizer.padding_side

'left'

# Calculate rouge

In [90]:
# Import required libraries
import torch
from tqdm import tqdm

# Get the first 100 samples from test dataset
test_sample = test_dataset.select(range(100))

# Extract validation inputs and references
val_inputs = [example["question"] for example in test_sample]

references = [example["answer"] for example in test_sample]
ref_reasonings = [example["explanation"] for example in test_sample]

# Generate predictions with batch processing
print("Generating predictions...")
predictions = generate_batch_responses(val_inputs, model, tokenizer)

# Example
print("\nFirst 3 examples:")
for i in range(3):
    print(f"\nInput: {val_inputs[i]}")
    print(f"Prediction Reasoning: {predictions[i]['reasoning']}")
    print(f"Prediction Answer: {predictions[i]['answer']}")
    print(f"Reference: {references[i]}")
    print("-" * 80)

Generating predictions...


Generating responses: 100%|██████████| 5/5 [02:36<00:00, 31.36s/it]


First 3 examples:

Input: Vịt của Janet đẻ được 16 quả trứng mỗi ngày. Cô ấy ăn 3 quả vào bữa sáng và nướng bánh muffin cho bạn bè mỗi ngày với 4 quả. Cô ấy bán số trứng còn lại tại chợ nông sản hàng ngày với giá $2 mỗi quả trứng vịt tươi. Cô ấy kiếm được bao nhiêu đô la mỗi ngày tại chợ nông sản?
Prediction Reasoning: Đầu tiên, chúng ta tính số quả trứng mà cô ấy giữ lại mỗi ngày bằng cách trừ số quả trứng cô ấy đã ăn và số quả trứng cô ấy làm bánh từ tổng số trứng cô ấy nhận được: 16 quả trứng - 3 quả trứng - 4 quả trứng = 9 quả trứng. Tiếp theo, chúng ta tính số tiền cô ấy kiếm được bằng cách nhân số quả trứng cô ấy giữ lại với giá của mỗi quả trứng: 9 quả trứng * $2/quả trứng = $18.
Prediction Answer: 18
Reference: Janet bán 9 quả trứng vịt mỗi ngày.
Cô ấy kiếm được 18 đô la mỗi ngày tại chợ nông sản.
### 18
--------------------------------------------------------------------------------

Input: Một chiếc áo choàng cần 2 cuộn sợi màu xanh và một nửa số đó là sợi màu trắng. Tổng cộ

In [92]:
predictions

[{'reasoning': 'Đầu tiên, chúng ta tính số quả trứng mà cô ấy giữ lại mỗi ngày bằng cách trừ số quả trứng cô ấy đã ăn và số quả trứng cô ấy làm bánh từ tổng số trứng cô ấy nhận được: 16 quả trứng - 3 quả trứng - 4 quả trứng = 9 quả trứng. Tiếp theo, chúng ta tính số tiền cô ấy kiếm được bằng cách nhân số quả trứng cô ấy giữ lại với giá của mỗi quả trứng: 9 quả trứng * $2/quả trứng = $18.',
  'answer': '18',
  'raw': 'Đầu tiên, chúng ta tính số quả trứng mà cô ấy giữ lại mỗi ngày bằng cách trừ số quả trứng cô ấy đã ăn và số quả trứng cô ấy làm bánh từ tổng số trứng cô ấy nhận được: 16 quả trứng - 3 quả trứng - 4 quả trứng = 9 quả trứng. Tiếp theo, chúng ta tính số tiền cô ấy kiếm được bằng cách nhân số quả trứng cô ấy giữ lại với giá của mỗi quả trứng: 9 quả trứng * $2/quả trứng = $18.\n### 18'},
 {'reasoning': 'Đầu tiên, tính số cuộn sợi màu trắng: 2 cuộn sợi / 2 = 1 cuộn sợi\nSau đó, cộng hai số lượng đó để tìm tổng số cuộn sợi: 1 cuộn sợi + 2 cuộn sợi = 3 cuộn sợi',
  'answer': '3',


In [93]:
# Calculate ROUGE scores
print("\nCalculating ROUGE scores on reasoning...")
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize
rouge_scores = {
    'rouge1': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
    'rouge2': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
    'rougeL': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0}
}

# Loop
for pred, ref in tqdm(zip(predictions, ref_reasonings), total=len(predictions), desc="Computing ROUGE"):
    pred_reasoning = pred["reasoning"]
    scores = scorer.score(ref, pred_reasoning)
    
    for metric in rouge_scores.keys():
        rouge_scores[metric]['precision'] += scores[metric].precision
        rouge_scores[metric]['recall'] += scores[metric].recall
        rouge_scores[metric]['fmeasure'] += scores[metric].fmeasure

# Average
n = len(predictions)
for metric in rouge_scores.keys():
    for key in rouge_scores[metric].keys():
        rouge_scores[metric][key] /= n

# Print
print("\nROUGE Scores:")
for metric, scores in rouge_scores.items():
    print(f"\n{metric}:")
    for key, value in scores.items():
        print(f"  {key}: {value:.4f}")


Calculating ROUGE scores on reasoning...


Computing ROUGE: 100%|██████████| 100/100 [00:00<00:00, 653.08it/s]


ROUGE Scores:

rouge1:
  precision: 0.6416
  recall: 0.6285
  fmeasure: 0.5971

rouge2:
  precision: 0.3492
  recall: 0.3439
  fmeasure: 0.3259

rougeL:
  precision: 0.4649
  recall: 0.4525
  fmeasure: 0.4308


# Calculate sacrebleu score

In [94]:
# Calculate sacrebleu score
import sacrebleu

# Prepare references and predictions for sacrebleu
pred_texts = [pred['reasoning'] for pred in predictions] 
refs_for_sacrebleu = [[ref] for ref in ref_reasonings]

# Calculate sacrebleu
print("\nCalculating BLEU score on reasoning...")
bleu = sacrebleu.corpus_bleu(pred_texts, refs_for_sacrebleu)
print(f"\nSacreBLEU Score: {bleu.score:.4f}")


Calculating BLEU score on reasoning...

SacreBLEU Score: 21.1412


# Calculate Exact Match Score:

In [95]:
# Calculate Exact Match Score
exact_match_count = 0

for pred, ref_ans in zip(predictions, references):
    if pred['answer'].strip() == ref_ans.split('###')[-1].strip():
        exact_match_count += 1

exact_match_score = exact_match_count / len(predictions)

print(f"\nExact Match Score: {exact_match_score:.4f}")


Exact Match Score: 0.4700


In [96]:
exact_match_count

47

# Save all metrics

In [97]:
# Save all metrics
import json
from datetime import datetime

# Create a summary dictionary of all metrics
evaluation_metrics = {
    'perplexity': perplexity,
    'sacrebleu': bleu.score,
    'rouge_scores': rouge_scores,
    'exact_match_score': exact_match_score,
}

# Prepare metrics with timestamp for saving
metrics_filename = "./logs/qwen-2.5-3b-instruct-math-qa.json"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

metrics_to_save = {
    'timestamp': timestamp,
    'model_name': "./logs/qwen-2.5-3b-instruct-math-qa",
    'metrics': evaluation_metrics
}

# Print summary before saving
print(f"Perplexity: {evaluation_metrics['perplexity']:.4f}")
print(f"SacreBLEU Score: {evaluation_metrics['sacrebleu']:.4f}")
print("\nROUGE Scores Summary:")
for metric, scores in evaluation_metrics['rouge_scores'].items():
    print(f"{metric} F1: {scores['fmeasure']:.4f}")
print(f"\nExact Match Score: {evaluation_metrics['exact_match_score']:.4f}")

# Save to file
with open(metrics_filename, 'w', encoding='utf-8') as f:
    json.dump(metrics_to_save, f, indent=2, ensure_ascii=False)

print(f"\nMetrics saved to {metrics_filename}")

Perplexity: 1.7495
SacreBLEU Score: 21.1412

ROUGE Scores Summary:
rouge1 F1: 0.5971
rouge2 F1: 0.3259
rougeL F1: 0.4308

Exact Match Score: 0.4700

Metrics saved to ./logs/qwen-2.5-3b-instruct-math-qa.json
